In [1]:
#Dependencies
import pandas as pd
from pandas import DataFrame

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [2]:
df = pd.read_csv("Data skripsi/fix hasil label/aspek_labeling_1.csv")
df

,Learnability,Efficiency,Memorability,Errors,Satisfaction,Ulasan
0,pos,pos,pos,pos,pos,Aplikasi ini sangat membantu saya saat pandemi...
1,pos,pos,pos,pos,pos,Memesan perjalanan lebih mudah
2,NaN,pos,NaN,NaN,pos,"KAI smakin bagus pelayananya, krena situasi Pa..."
3,NaN,NaN,NaN,NaN,pos,Kedepannya lebih menyenangkan lagi
4,pos,pos,pos,NaN,pos,Sy banyak merasa terbantu
...,...,...,...,...,...,...
95,NaN,neg,NaN,neg,neg,"Parah...g kek dlu,skrng ssah bgt mo pesen lemo..."
96,neg,NaN,neg,NaN,neg,Gw cuma kasihan sama orang-orang tua dari desa...
97,NaN,NaN,NaN,NaN,neg,Ya Allah kirain saya aja yg gak dapat verifika...
98,NaN,NaN,NaN,NaN,neg,Kenapa pembatalan tiket kereta tidak bisa dila...


In [3]:
replacers = {'sy':'saya','gak':'enggak','ssah':'susah','bgt':'banget','skrng':'sekarang','yg':'yang','gw':'gue',
             'baguuus':'bagus','mmbantu':'membantu'}


def word_normalization(doc):
    for key in replacers.keys():
        doc = doc.replace(key,replacers[key])
    return doc

In [4]:
def words_normalization2(sentence, dictionary):
    return " ".join([dictionary.get(w,w) for w in sentence.split()])

In [5]:
#clean the data
stop = set(stopwords.words('indonesian'))
exclude = set(string.punctuation + '1234567890')

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def preprocess1(text):
    case_folded = text.lower()
    symbol_removed = ''.join(ch for ch in case_folded if ch not in exclude)
    word_normalized = words_normalization2(symbol_removed, replacers)
    tokenized = word_tokenize(word_normalized)
    #stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    #stemmed = ' '.join([stemmer.stem(word) for word in num_and_punc_free.split()])
    
    return tokenized

In [6]:
df['preprocessed1_ulasan'] = df['Ulasan'].apply(preprocess1)

In [7]:
df['preprocessed1_ulasan']

0     [aplikasi, ini, sangat, membantu, saya, saat, ...
1                   [memesan, perjalanan, lebih, mudah]
2     [kai, smakin, bagus, pelayananya, krena, situa...
3               [kedepannya, lebih, menyenangkan, lagi]
4                      [saya, banyak, merasa, terbantu]
                            ...                        
95    [parahg, kek, dluskrng, susah, banget, mo, pes...
96    [gue, cuma, kasihan, sama, orangorang, tua, da...
97    [ya, allah, kirain, saya, aja, yang, enggak, d...
98    [kenapa, pembatalan, tiket, kereta, tidak, bis...
99                   [bagus, buat, membantu, informasi]
Name: preprocessed1_ulasan, Length: 100, dtype: object

In [8]:
from nltk.tag import CRFTagger
ct = CRFTagger()
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')

pos_ulasan = ct.tag_sents(df.preprocessed1_ulasan)

In [9]:
pos_ulasan

[[('aplikasi', 'NN'),
  ('ini', 'PR'),
  ('sangat', 'RB'),
  ('membantu', 'VB'),
  ('saya', 'PRP'),
  ('saat', 'NN'),
  ('pandemi', 'NN'),
  ('ini', 'PR'),
  ('karena', 'SC'),
  ('dengan', 'IN'),
  ('aplikasi', 'NN'),
  ('saya', 'PRP'),
  ('merasa', 'VB'),
  ('lebih', 'RB'),
  ('terbantu', 'VB'),
  ('dan', 'CC'),
  ('lebih', 'RB'),
  ('di', 'IN'),
  ('mudahkan', 'VB'),
  ('untuk', 'IN'),
  ('memesan', 'NN'),
  ('tiket', 'NN'),
  ('kereta', 'NN'),
  ('api', 'NN'),
  ('terimakasih', 'MD'),
  ('kak', 'VB'),
  ('access', 'FW')],
 [('memesan', 'NN'), ('perjalanan', 'NN'), ('lebih', 'RB'), ('mudah', 'JJ')],
 [('kai', 'VB'),
  ('smakin', 'RB'),
  ('bagus', 'JJ'),
  ('pelayananya', 'RB'),
  ('krena', 'VB'),
  ('situasi', 'NN'),
  ('pandemi', 'NN'),
  ('covid', 'NN'),
  ('sayang', 'NN'),
  ('agak', 'RB'),
  ('tergesa', 'NN'),
  ('urus', 'FW'),
  ('rapid', 'FW'),
  ('tes', 'FW'),
  ('di', 'IN'),
  ('stasiun', 'CD')],
 [('kedepannya', 'RB'),
  ('lebih', 'RB'),
  ('menyenangkan', 'VB'),
  ('lagi',

In [10]:
noun_data = list()
for i,text in enumerate(pos_ulasan, start=0):
    temp = list()
    for term in text:
        if term[1] == 'VB' or term[1] == 'NN' or term[1] == 'JJ' or term[1] == 'RB' or term[1] == 'NEG' or term[1] == 'NNP':
        #if term[1] == 'NNP' or term[1] == 'NN':
            temp.append(term[0])
    noun_data.append(temp)

df['pos_ulasan'] = df['Ulasan']

for i in range(len(df['pos_ulasan'])):
    df['pos_ulasan'][i] = noun_data[i]

In [11]:
#def neg_handler(doc):
 #   for key in neg_word.keys():
  #      doc = doc.replace(key,neg_word[key])
   # return doc

def preprocess2(text):
    text = TreebankWordDetokenizer().detokenize(text)
    stop_removed = ' '.join([word for word in text.split() if word not in stop])
    stemmed = ' '.join([stemmer.stem(word) for word in stop_removed.split()])
    tokenized = word_tokenize(stemmed)
    return tokenized

In [12]:
df['preprocessed2_ulasan'] = df['pos_ulasan'].apply(preprocess2)

In [13]:
df['pos_ulasan']

0     [aplikasi, sangat, membantu, saat, pandemi, ap...
1                   [memesan, perjalanan, lebih, mudah]
2     [kai, smakin, bagus, pelayananya, krena, situa...
3               [kedepannya, lebih, menyenangkan, lagi]
4                                              [merasa]
                            ...                        
95    [parahg, kek, dluskrng, susah, bsbknya, tambah...
96    [gue, cuma, kasihan, sama, orangorang, tua, de...
97    [ya, allah, kirain, aja, enggak, verifikasi, e...
98    [pembatalan, tiket, kereta, tidak, dilakukan, ...
99                   [bagus, buat, membantu, informasi]
Name: pos_ulasan, Length: 100, dtype: object

In [69]:
import sklearn as sk
import math

def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    return unique_list

def get_unique_token(corpus):
    total = list()
    for doc in corpus:
        total = set(total).union(set(doc))
    return unique(total)

first = df['preprocessed2_ulasan'][0]
second = df['preprocessed2_ulasan'][1]
third = df['preprocessed2_ulasan'][2]

unique_token = get_unique_token(df.preprocessed2_ulasan)

#Now lets add a way to count the words using a dictionary key-value pairing for both sentences
wordDictA = dict.fromkeys(unique_token, 0) 
wordDictB = dict.fromkeys(unique_token, 0)
wordDictC = dict.fromkeys(unique_token, 0)

for word in first:
    wordDictA[word]+=1
    
for word in second:
    wordDictB[word]+=1
    
for word in third:
    wordDictC[word]+=1
    
#put them in a dataframe and then view the result:
pd.DataFrame([wordDictA, wordDictB,  wordDictC])

,salah,aplikasi,tingkat,komplain,membuangbuang,lelah,login,email,ribut,tiket,...,ajak,terima,tutorial,lupa,bayar,pesan,trip,nomor,aneh,so
0,1,0,0,1,1,1,2,1,1,0,...,1,0,0,1,0,0,0,1,1,0
1,1,2,1,0,0,0,0,0,0,3,...,0,0,1,0,1,0,1,0,0,1
2,0,2,0,0,0,0,0,0,0,1,...,0,1,0,0,1,1,0,0,0,0


In [73]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict
#running our sentences through the tf function:
tfFirst = computeTF(wordDictA, first)
tfSecond = computeTF(wordDictB, second)
tfThird = computeTF(wordDictC, third)
#Converting to dataframe for visualization
tf_df = pd.DataFrame([tfFirst, tfSecond, tfThird])

In [75]:
tf_df

,salah,aplikasi,tingkat,komplain,membuangbuang,lelah,login,email,ribut,tiket,...,ajak,terima,tutorial,lupa,bayar,pesan,trip,nomor,aneh,so
0,0.035714,0.000000,0.000000,0.035714,0.035714,0.035714,0.071429,0.035714,0.035714,0.000000,...,0.035714,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.035714,0.035714,0.000000
1,0.045455,0.090909,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.136364,...,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.045455
2,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.066667,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000


In [76]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict
#inputing our sentences in the log file
idfs = computeIDF([wordDictA, wordDictB, wordDictC])
#The actual calculation of TF*IDF from the table above:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf
#running our two sentences through the IDF:
idfFirst = computeTFIDF(tfFirst, idfs)
idfSecond = computeTFIDF(tfSecond, idfs)
idfThird = computeTFIDF(tfThird, idfs)
#putting it in a dataframe
idf= pd.DataFrame([idfFirst, idfSecond, idfThird])

In [77]:
idf

,salah,aplikasi,tingkat,komplain,membuangbuang,lelah,login,email,ribut,tiket,...,ajak,terima,tutorial,lupa,bayar,pesan,trip,nomor,aneh,so
0,0.006289,0.000000,0.000000,0.01704,0.01704,0.01704,0.03408,0.01704,0.01704,0.000000,...,0.01704,0.000000,0.000000,0.01704,0.000000,0.000000,0.000000,0.01704,0.01704,0.000000
1,0.008004,0.016008,0.021687,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.024012,...,0.00000,0.000000,0.021687,0.00000,0.008004,0.000000,0.021687,0.00000,0.00000,0.021687
2,0.000000,0.023479,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.011739,...,0.00000,0.031808,0.000000,0.00000,0.011739,0.031808,0.000000,0.00000,0.00000,0.000000


In [78]:
tf_idf = tf_df * idf

In [79]:
tf_idf

,salah,aplikasi,tingkat,komplain,membuangbuang,lelah,login,email,ribut,tiket,...,ajak,terima,tutorial,lupa,bayar,pesan,trip,nomor,aneh,so
0,0.000225,0.000000,0.000000,0.000609,0.000609,0.000609,0.002434,0.000609,0.000609,0.000000,...,0.000609,0.000000,0.000000,0.000609,0.000000,0.000000,0.000000,0.000609,0.000609,0.000000
1,0.000364,0.001455,0.000986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003274,...,0.000000,0.000000,0.000986,0.000000,0.000364,0.000000,0.000986,0.000000,0.000000,0.000986
2,0.000000,0.003131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000783,...,0.000000,0.002121,0.000000,0.000000,0.000783,0.002121,0.000000,0.000000,0.000000,0.000000


In [99]:
#first step is to import the library
from sklearn.feature_extraction.text import TfidfVectorizer
#for the sentence, make sure all words are lowercase or you will run into error. for simplicity, I just made the same sentence all lowercase
firstV = 'Ibu pergi ke pasar'
secondV = 'Saya pergi ke sekolah bersama ibu'
thirdV = 'Bakso di pasar pagi lebih enak dari bakso di pasar malam'

def get_corpus(corpus):
    total = []
    for doc in corpus:
        total = set(total).union(set(doc))
    return total

get_corpus(df.preprocessed2_ulasan)


{'ajak',
 'aneh',
 'anjing',
 'aplikasi',
 'banget',
 'bayar',
 'beli',
 'buka',
 'dealmedan',
 'dunia',
 'email',
 'gilir',
 'ha',
 'jaya',
 'keren',
 'kereta',
 'komplain',
 'konfirmasi',
 'lelah',
 'lihat',
 'login',
 'lupa',
 'masuk',
 'membuangbuang',
 'mengulangulang',
 'mudah',
 'nomor',
 'nya',
 'pakai',
 'perkeretaapian',
 'pesan',
 'ptkai',
 'ribut',
 'salah',
 'sandi',
 'saran',
 'so',
 'staff',
 'temu',
 'terima',
 'tiket',
 'tingkat',
 'trip',
 'tulis',
 'tutorial',
 'ulas',
 'verifikasi'}

In [ ]:
#calling the TfidfVectorizer
vectorize= TfidfVectorizer()
#fitting the model and passing our sentences right away:
response= vectorize.fit_transform([firstV, secondV, thirdV])
print(response)